In [1]:
import numpy as np
import pandas as pd
#import cPickle
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os


from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed, SpatialDropout1D
from keras.models import Model
from keras.layers.normalization import BatchNormalization

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers, regularizers, constraints

/anaconda3/envs/textClassifier/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [15]:
MAX_SENT_LENGTH = 60
MAX_SENTS = 15
MAX_NB_WORDS = 1000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.0


def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\"", "", string)
    return string.strip().lower()

In [16]:
data_train = pd.read_csv('document similarity/document_similarity.csv', sep=',')
print(data_train.shape)

data_train.dropna(inplace=True)

#data_train['cat_id'] = 0
#for i,row in data_train.iterrows():
#    if row['category'] == 'unrest':
#        data_train.set_value(i,'cat_id',0)
#    else:
#        data_train.set_value(i,'cat_id',1)

(9, 5)


In [17]:
data_train.head()

,Unnamed: 0,category,content,doc2vec,doc2veccosine
0,0,movie,Padmaavat: Why a Bollywood epic has sparked fi...,[ -9.05430984 1.71762216 1.08929253 -3.22...,[ 1. 0.94052142 0.9796434 0.926054...
1,1,lokpal,Anna Hazare on fast-unto-death demanding Jan L...,[ -4.08943748e+00 3.65195364e-01 4.3100047...,[ 0.94052142 1. 0.97000271 0.940714...
2,2,movie,A Film Has Inflamed Indians. But Moviegoers Mo...,[ -8.49335098 0.55525446 1.04886711 -3.75...,[ 0.9796434 0.97000271 1. 0.946262...
3,3,movie,Indian police detain dozens protesting against...,[ -6.10597134 0.87642241 0.60653818 -3.09...,[ 0.92605472 0.9407143 0.94626272 0.999999...
4,4,lokpal,Thousands Back Antigraft Hunger Strike in New ...,[-11.33139229 2.65446663 1.2543447 -0.94...,[ 0.8824504 0.86918712 0.90175384 0.953397...


In [18]:
from nltk import tokenize

reviews = []
labels = []
texts = []
i=0

for idx in data_train['Unnamed: 0']:
    i = idx
    text = BeautifulSoup(data_train.content[idx])
    text = clean_str(text.get_text())
    texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    reviews.append(sentences)

    #labels.append(data_train.cat_id[idx])

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)

data = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

/anaconda3/envs/textClassifier/lib/python3.6/site-packages/keras_preprocessing/text.py:175: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [19]:
for i, sentences in enumerate(reviews):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTS:
            # convert sentence to words
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
                # If the number of words in a sentence is less than 100 and it's index is < 20000 then assign it.
                if k < MAX_SENT_LENGTH and tokenizer.word_index[word] < MAX_NB_WORDS:
                    data[i, j, k] = tokenizer.word_index[word]
                    k = k + 1

word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

#labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
#print('Shape of label tensor:', labels.shape)

#indices = np.arange(data.shape[0])
#np.random.shuffle(indices)
#data = data[indices]
#labels = labels[indices]
#nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

Total 1553 unique tokens.
Shape of data tensor: (9, 15, 60)


In [7]:
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of positive and negative reviews in traing and validation set')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

GLOVE_DIR = "/Users/venkatakrishnamohansunkara/Desktop/DM/textClassifier"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors.' % len(embeddings_index))

Number of positive and negative reviews in traing and validation set
[35262. 35140.]
[8757. 8843.]
Total 400000 word vectors.


In [20]:
test_data = data

In [8]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
         #words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        embedding_matrix[i] = 100*[0]

In [9]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SENT_LENGTH,
                            trainable=True,
                            mask_zero=True)

In [10]:
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

    
class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        
        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        #a = K.exp(ait)
        a = K.softmax(ait)
        # apply mask after the exp. will be re-normalized next
        #if mask is not None:
        #    # Cast the mask to floatX to avoid float64 upcasting in theano
        #    a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [11]:
input_layer = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH),dtype='int32')
sentence_input = Input(shape=(MAX_SENT_LENGTH,),dtype='int32')
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SENT_LENGTH,
                            trainable=True,
                            mask_zero=True)(sentence_input)
drop1 = SpatialDropout1D(0.3)(embedding_layer)
sent_lstm = Bidirectional(LSTM(100, name='blstm_1',
        activation='tanh',
        recurrent_activation='hard_sigmoid',
        recurrent_dropout=0.0,
        dropout=0.4,
        kernel_initializer='glorot_uniform',
        return_sequences=True),
        merge_mode='concat')(drop1)

sent_att_layer = AttentionWithContext()(sent_lstm)
sentEncoder = Model(sentence_input, sent_att_layer)
sentEncoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 60)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 60, 100)           40139200  
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 60, 100)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 60, 200)           160800    
_________________________________________________________________
attention_with_context_1 (At (None, 200)               40400     
Total params: 40,340,400
Trainable params: 40,340,400
Non-trainable params: 0
_________________________________________________________________


In [12]:
textEncoder = TimeDistributed(sentEncoder)(input_layer)
drop2 = Dropout(0.4)(textEncoder)

lstm_1 = Bidirectional(LSTM(100, name='blstm_2',
        activation='tanh',
        recurrent_activation='hard_sigmoid',
        recurrent_dropout=0.0,
        dropout=0.4,
        kernel_initializer='glorot_uniform',
        return_sequences=True),
        merge_mode='concat')(drop2)
lstm_1 = BatchNormalization()(lstm_1)
att_layer = AttentionWithContext()(lstm_1)
drop3 = Dropout(0.5)(att_layer)
predictions = Dense(2, activation='softmax')(drop3)
model = Model(inputs=input_layer, outputs=predictions)
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])
#print("model fitting - Hierachical attention network")
#model.fit(x_train, y_train, validation_data=(x_val, y_val),
#          nb_epoch=100, batch_size=50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 15, 60)            0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 15, 200)           40340400  
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 200)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 15, 200)           240800    
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 200)           800       
_________________________________________________________________
attention_with_context_2 (At (None, 200)               40400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
__________

In [13]:
sentEncoder.load_weights('/Users/venkatakrishnamohansunkara/Desktop/DM/sent_encoder_lstm.h5')

In [14]:
model.load_weights('/Users/venkatakrishnamohansunkara/Desktop/DM/model_lstm.h5')

In [ ]:
# Get the 7000 unrest articles filtered and test it

In [21]:
data_test = pd.read_csv('document similarity/document_similarity.csv')

In [22]:
data_test['HATT embeddings'] = ''

In [33]:
get_embeddings = K.function([model.layers[0].input, K.learning_phase()], [model.layers[6].output])
for i in range(test_data.shape[0]):
    location = i
    embeddings = get_embeddings([test_data[location],0])
    data_test.set_value(i,'HATT embeddings',np.asarray(embeddings[0][0]))

/anaconda3/envs/textClassifier/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """


In [26]:
location = 0
# Get the prediction
get_output = K.function([model.layers[0].input, K.learning_phase()], [model.layers[7].output])
predictions = get_output([test_data[location],0])
print(predictions)
print(y_train[location])
# Get output before the attention layer
get_layer_output = K.function([model.layers[0].input, K.learning_phase()], [model.layers[4].output])
out = get_layer_output([test_data[location],0])  # test mode
print(out[0].shape)

[array([[8.284906e-11, 1.000000e+00]], dtype=float32)]
[1. 0.]
(1, 15, 200)


In [88]:
y_train.shape

(70402, 2)

In [89]:
#all(labels == [1,0])

In [90]:
labels = y_train[location]
if all(labels == [1,0]):
    label = 'unrest';
else:
    label = 'other'

In [91]:
labels

array([1., 0.], dtype=float32)

In [92]:
if predictions[0][0][1]>predictions[0][0][0]:
    prediction = 'other';
else:
    prediction = 'unrest';

In [93]:
# Get the attention weights of the sentence network.
eij = np.tanh(np.dot(out[0],model.layers[5].get_weights()[0]))
t = np.dot(eij,model.layers[5].get_weights()[2])
ai = np.exp(t)
weights = ai/np.sum(ai)

In [94]:
# 15 weights for 15 sentences.
weights.shape
#np.sum(out[0]*weights,axis=1).shape

(1, 15)

In [95]:
# Get the attention weights of the word network for every sentence.
get_layer_op_words = K.function([sentEncoder.layers[0].input, K.learning_phase()], [sentEncoder.layers[3].output])
op_words = get_layer_op_words([x_train[location],0])
print(op_words[0].shape)       

(15, 60, 200)


In [96]:
weight_all_words = []
for i in range(MAX_SENTS):
    eij_words = np.tanh(np.dot(op_words[0][i],sentEncoder.layers[4].get_weights()[0]))
    t_words = np.dot(eij_words,sentEncoder.layers[4].get_weights()[2])
    ai_words = np.exp(t_words)
    weights_words = ai_words/np.sum(ai_words)
    weight_all_words.append(weights_words)

In [97]:
# 60 weights for 60 words 
len(weight_all_words)

15

In [98]:
id2word = {v: k for k, v in word_index.items()}
with open("Recent_Visualizations/visualization_test.html", "w") as html_file:
    html_file.write('<!DOCTYPE html>\n')
    html_file.write('<html>\n')
    html_file.write('<body>\n')
    sent_no = 0
    html_file.write('<p>\n')
    html_file.write('<h3>Prediction: %s</h3>\n' %prediction)
    html_file.write('<h3>Label: %s</h3>\n' %label)
    html_file.write('</p>\n')
    for i in x_train[location]:
        # Iterate through the words in the sentence
        weights_words = weight_all_words[sent_no]
        sent = ''
        word_no =0
        html_file.write('<font style="background-color: rgba(255, 0, 0, %f)">%s</font>\n' %(weights[0][sent_no],str(weights[0][sent_no])))
        for j in i:
            if j!=0:
                alpha = weights_words[word_no]*40
                word = id2word[j]
                html_file.write('<font style="background-color: rgba(0, 0, 255, %f)">%s</font>\n' % (alpha, word))
                sent+=id2word[j]+' '+str(weights_words[word_no])+' ' 
            word_no+=1
        html_file.write('<br />\n')
        print(str(weights[0][sent_no])+'---'+sent+'.')
        sent_no+=1
    #print('<br />\n')
    #print('<br />\n')
    #print('<h5> Original Document<h5>\n')
    #print('<p>\n')
    #print('%s\n' %data_train.iloc[location]['content'])
    #print('</p>\n')
    html_file.write('</body>\n')
    html_file.write('</html>\n')

0.10022662---coimbatore 0.06340147 unfazed 0.023951871 threats 0.02011564 sasikala 0.038772874 family 0.014973568 united 0.013860773 edappadi 0.014677104 k 0.01206006 palaniswamy 0.019769648 faction 0.012154868 dared 0.019205896 ttv 0.018676732 dhinakaran 0.007816201 bring 0.011013801 government 0.013180024 .
0.058086295---mla 0.16205435 v 0.014369314 c 0.029780287 arukutty 0.031516973 said 0.022466492 sasikala 0.025386395 family 0.0066732434 members 0.015372422 would 0.009487164 allowed 0.008164034 continue 0.013582026 aiadmk 0.0340802 even 0.012857487 government 0.01051414 falls 0.007902034 .
0.079827234---cost 0.005626709 sasikala 0.009243172 family 0.0023460703 members 0.0076649003 entertained 0.037988238 aiadmk 0.01693854 .
0.3140566---even 0.007824148 government 0.008271808 falls 0.0052241683 allowed 0.0060349302 party 0.008252758 arukutty 0.02236415 told 0.010345576 reporters 0.008443309 coimbatore 0.016609147 wednesday 0.013680717 .
0.14848755---let 0.009695693 dhinakaran 0.005

In [ ]:
sentEncoder.layers[4]

In [ ]:
x_train[0].shape

In [ ]:
data_train.iloc[location]['Content']

In [ ]:
data.shape

In [ ]:
model.evaluate(data,labels)

In [ ]:
labels = np.tile([0,1],(7061,1))

In [ ]:
labels[2]

In [ ]:
#from keras.utils import plot_model
#plot_model(model, to_file='model.png')

In [39]:
data_test.to_csv('document similarity/document_similarity_hatt.csv')

In [40]:
data_test['hattcosine'] = ''

In [41]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(a,b):
    return dot(a, b)/(norm(a)*norm(b))

In [42]:
df = data_test

In [43]:
for i,row1 in df.iterrows():
    l= []
    for j,row2 in df.iterrows():
        cs = cos_sim(row1['HATT embeddings'],row2['HATT embeddings'])
        l+=[cs]
    df.set_value(i,'hattcosine',np.asarray(l))

/anaconda3/envs/textClassifier/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [46]:
for i,row in df.iterrows():
    print(i);
    r = np.sort(row['hattcosine'])[::-1][0:9]
    r2 = np.argsort(row['hattcosine'])[::-1][0:9]
    print(r)
    print(r2)

0
[1.0000001  0.92749    0.90885884]
[0 7 6]
1
[1.0000001  0.81181544 0.78600794]
[1 8 0]
2
[1.         0.71136224 0.5640011 ]
[2 1 8]
3
[1.0000001 0.9190015 0.911249 ]
[3 6 8]
4
[0.99999994 0.7212733  0.7097385 ]
[4 7 6]
5
[0.9999999  0.7778364  0.76642966]
[5 8 1]
6
[0.9999999  0.9190015  0.90885884]
[6 3 0]
7
[1.0000001 0.92749   0.9024751]
[7 0 6]
8
[0.9999999 0.911249  0.8382874]
[8 3 7]
